# Radial Basis Function Neural Network

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from itertools import product
import pickle

import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt

from system_identification.rbfnn_model import RadialBasisFunctionNeuralNetworkModel

## 1D

In [3]:
def foo(x):
    return -x**2


inputs = np.linspace(-1, 1, 1000)
reference_outputs = np.array(list(map(foo, inputs)))
reference_outputs_noisy = reference_outputs + np.random.normal(0, 0.1, size=reference_outputs.shape)

inputs = inputs.reshape(-1, 1, 1)
reference_outputs = reference_outputs.reshape(-1, 1, 1)
reference_outputs_noisy = reference_outputs_noisy.reshape(-1, 1, 1)

### Least squares

In [4]:
model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
    n_inputs=1,
    grid_size=[9],
    input_range=np.array([(-1, 1)]),
    rbf_width=1.75,
    rbf_amplitude=1,
)
model

<RadialBasisFunctionNeuralNetwork
 n_inputs=1
 n_hidden=9
 n_outputs=1>

In [5]:
model.train(inputs, reference_outputs_noisy)

In [6]:
fig = plt.figure()
outputs = model.evaluate(inputs)
plt.plot(inputs.squeeze(), reference_outputs.squeeze())
plt.plot(inputs.squeeze(), reference_outputs_noisy.squeeze(), ".", markersize=1)
plt.plot(inputs.squeeze(), outputs.squeeze())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Levenberg-Marquardt training

In [4]:
model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
    n_inputs=1,
    grid_size=[3],
    input_range=np.array([(-1, 1)]),
    rbf_width=3,
    rbf_amplitude=1,
)

In [5]:
model

<RadialBasisFunctionNeuralNetwork
 n_inputs=1
 n_hidden=3
 n_outputs=1>

In [6]:
model.train(
    inputs,
    reference_outputs,
    epochs=1000,
    goal=1e-6,
    train_log_freq=1,
    method="trainlm",
    mu=10.,
    alpha=0.95
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
fig = plt.figure()
outputs = model.evaluate(inputs)
plt.plot(inputs.squeeze(), reference_outputs.squeeze())
plt.plot(inputs.squeeze(), reference_outputs_noisy.squeeze(), ".", markersize=1)
plt.plot(inputs.squeeze(), outputs.squeeze())

fig, axs = plt.subplots(3, 1, figsize=(8, 6))
abs(model.training_log.error_training_data).mean("i").plot(ax=axs[0])
axs[0].set_yscale("log")
model.training_log.error_training_jb.plot(ax=axs[1])
model.training_log.error_training_jbp.plot(ax=axs[2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2D

In [10]:
def foo(x):
    return -x[0]**2 + np.sin(x[1]*2)


resolution = 100
input_range = np.array([(-1, 1), (-3, 5)])
inputs = np.array(tuple(product(np.linspace(*input_range[0, :], resolution),
                                np.linspace(*input_range[1, :], resolution))))

reference_outputs = np.array(list(map(foo, inputs)))
reference_outputs_noisy = reference_outputs + (np.random.random(reference_outputs.shape) - 0.5) * 0.1

inputs = inputs[..., None]
reference_outputs = reference_outputs.reshape(-1, 1, 1)
reference_outputs_noisy = reference_outputs_noisy.reshape(-1, 1, 1)

### Least squares

In [10]:
model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
    n_inputs=2,
    grid_size=[5, 15],
    input_range=input_range,
    rbf_width=1.75,
    rbf_amplitude=1,
)
model

<RadialBasisFunctionNeuralNetwork
 n_inputs=2
 n_hidden=75
 n_outputs=1>

In [11]:
model.train(inputs, reference_outputs)
output = model.evaluate(inputs)

In [12]:
fig = plt.figure(figsize=(4, 3))
plt.plot(model.weights_c[:, 0], model.weights_c[:, 1], "x")

fig = plt.figure(figsize=(4, 3))
ax = fig.add_subplot(projection='3d')
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], output.squeeze(), s=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Levenberg-Marquardt training

In [11]:
model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
    n_inputs=2,
    grid_size=[5, 7],
    input_range=input_range,
    rbf_width=1.75,
    rbf_amplitude=1,
)
model

<RadialBasisFunctionNeuralNetwork
 n_inputs=2
 n_hidden=35
 n_outputs=1>

In [80]:
# Pretrain the network using least squares.
model.train(inputs, reference_outputs_noisy)

In [12]:
model.train(
    inputs,
    reference_outputs_noisy,
    epochs=30,
    goal=1e-6,
    train_log_freq=1,
    method="trainlm",
    mu=10.,
    alpha=0.95
)

  0%|          | 0/30 [00:00<?, ?it/s]

In [82]:
model.training_log

<xarray.Dataset>
Dimensions:                (epoch: 30, i: 10000, j: 1, k: 35, l: 35, m: 2, n: 35, o: 2)
Coordinates:
  * epoch                  (epoch) int64 1 2 3 4 5 6 7 ... 24 25 26 27 28 29 30
Dimensions without coordinates: i, j, k, l, m, n, o
Data variables: (12/14)
    gradient               (epoch) float64 3.978e+04 4.502e+04 ... 5.336e+04
    error_training_data    (epoch, i) float64 -0.142 -0.09916 ... -0.04944
    error_validation_data  (epoch, j) float64 nan nan nan nan ... nan nan nan
    error_training_jb      (epoch) float64 131.5 60.79 0.7119 ... 356.6 370.3
    error_training_jbp     (epoch) float64 2.818e-29 6.301e-14 ... 3.924e-81
    error_training_skew    (epoch) float64 0.08382 0.1559 ... -0.00998 -0.007569
    ...                     ...
    error_validation_jbp   (epoch) float64 nan nan nan nan ... nan nan nan nan
    error_validation_skew  (epoch) float64 nan nan nan nan ... nan nan nan nan
    error_validation_kurt  (epoch) float64 nan nan nan nan ... nan nan nan nan
    weights_a              (epoch, k) float64 -1.184 -0.7737 ... -0.2823 -1.747
    weights_c              (epoch, l, m) float64 -1.228 -3.324 ... 1.26 5.201
    weights_w              (epoch, n, o) float64 1.608 1.719 ... 1.354 1.202

In [83]:
fig = plt.figure(figsize=(4, 3))
plt.plot(model.weights_c[:, 0], model.weights_c[:, 1], "x")

fig = plt.figure(figsize=(4, 3))
ax = fig.add_subplot(projection='3d')
output = model.evaluate(inputs)
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(inputs[:, 0, 0], inputs[:, 1, 0], output.squeeze(), s=0.1)

fig, axs = plt.subplots(3, 1, figsize=(8, 6))
abs(model.training_log.error_training_data).mean("i").plot(ax=axs[0])
axs[0].set_yscale("log")
model.training_log.error_training_jb.plot(ax=axs[1])
model.training_log.error_training_jbp.plot(ax=axs[2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Random within range RBF initialization.

In [34]:
model = RadialBasisFunctionNeuralNetworkModel.new_random_placement(
    n_inputs=2,
    n_hidden=4,
    input_range=input_range,
    width_range=(1.5, 2.5),
    amplitude_range=(0.5, 1.5),
)
model

<RadialBasisFunctionNeuralNetwork
 n_inputs=2
 n_hidden=4
 n_outputs=1>

In [35]:
# Check if the weights are within the given range.
assert (0.5 <= model.weights_a).all()
assert (1.5 >= model.weights_a).all()
assert (1.5 <= model.weights_w).all()
assert (2.5 >= model.weights_w).all()
assert (input_range[0, 0] <= model.weights_c[:, 0]).all()
assert (input_range[0, 1] >= model.weights_c[:, 0]).all()
assert (input_range[1, 0] <= model.weights_c[:, 1]).all()
assert (input_range[1, 1] >= model.weights_c[:, 1]).all()

## Pickling

In [3]:
model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
    n_inputs=1,
    grid_size=[9],
    input_range=np.array([(-1, 1)]),
    rbf_width=1.75,
    rbf_amplitude=1,
)

In [13]:
pickled = pickle.dumps(model)

In [14]:
unpickled = pickle.loads(pickled)

In [15]:
unpickled

<RadialBasisFunctionNeuralNetwork
 n_inputs=2
 n_hidden=35
 n_outputs=1>